In [25]:
from xarm.wrapper import XArmAPI
import time


SDK_VERSION: 1.13.30


In [26]:
# Replace '192.168.1.1' with the actual IP address of your xArm7
arm = XArmAPI('192.168.86.216')

ROBOT_IP: 192.168.86.216, VERSION: v1.10.0, PROTOCOL: V1, DETAIL: 7,7,XS1303,AC1300,v1.10.0, TYPE1300: [1, 1]
change protocol identifier to 3


ControllerError, code: 1
ControllerError had clean
ControllerError, code: 23
ControllerError had clean
ControllerError, code: 23
ControllerError had clean
ControllerError, code: 23
ControllerError had clean
ControllerError, code: 23
ControllerError had clean
ControllerError, code: 23
ControllerError had clean
ControllerError, code: 23
ControllerError had clean


In [27]:
# Connect to the xArm
arm.connect()

In [28]:
# Ensure the arm is powered on and in position control mode
arm.motion_enable(enable=True)
arm.set_mode(0)
arm.set_state(0)

0

In [40]:
code, current_pos = arm.get_position(is_radian=False)
print(f"Current position: {current_pos}")

Current position: [205.999969, -0.0, 253.954681, -173.981327, -12.013836, -23.946542]


In [33]:
new_pos = [305.518585, 9.993493, 541.88092, 0, -87.419379, -136.354903]

arm.set_position(*new_pos, speed=100, mvacc=1000, wait=True)

0

In [70]:
code, current_pos = arm.get_position(is_radian=False)
print(f"Current position: {current_pos}")

new_pos = [0 for _ in range(6)]
new_pos[4] = -20
if code == 0:
    arm.set_position(*new_pos, speed=100, mvacc=1000, wait=True, relative=True)
    
code, current_pos = arm.get_position(is_radian=False)
print(f"Current position: {current_pos}")
    

Current position: [267.570374, 8.021536, 434.706421, -176.334179, -70.336356, -3.771724]
Current position: [267.570374, 8.021536, 434.706421, -73.412166, -88.677366, -106.470226]


In [24]:
code, current_pos = arm.get_position(is_radian=False)
print(current_pos)

NameError: name 'arm' is not defined

In [63]:
code, current_pos = arm.get_position(is_radian=False)
print(f"Current position: {current_pos}")
print(f"Initial position: {initial_pos}")

Current position: [339.618286, 13.401848, 539.494507, -5.028278, -74.001109, -174.191603]
Initial position: [310.77179, 13.401848, 568.609253, -59.480124, -88.155229, -119.950433]


In [100]:
prev_pos

[28.847080998605932, 0.0, -29.115334135276584, 0.0, -14.999999999999996, 0.0]

[SDK][ERROR][2024-06-12 23:48:30][base.py:168] - - [report-socket] socket read timeout


In [1]:
from scipy.spatial.transform import Rotation as R
import numpy as np
r = R.from_euler('xyz', [0,15,0], degrees=True).as_matrix()

print(r)
theta = np.pi / 12

transformation_matrix1 = np.array([
    [ np.cos(theta), 0, np.sin(theta), 125],
    [            0, 1,            0, 0],
    [-np.sin(theta), 0, np.cos(theta), 95],
    [            0, 0,            0, 1]
])
print(transformation_matrix1)

[[ 0.96592583  0.          0.25881905]
 [ 0.          1.          0.        ]
 [-0.25881905  0.          0.96592583]]
[[  0.96592583   0.           0.25881905 125.        ]
 [  0.           1.           0.           0.        ]
 [ -0.25881905   0.           0.96592583  95.        ]
 [  0.           0.           0.           1.        ]]


In [87]:
prev_pos = np.zeros(6)

theta = -1 * np.pi / 12
# theta = 0

transformation_matrix1 = np.array([
    [ np.cos(theta), 0, np.sin(theta), 125],
    [            0, 1,            0, 0],
    [-np.sin(theta), 0, np.cos(theta), 95],
    [            0, 0,            0, 1]
])
print(transformation_matrix1)

[[  0.96592583   0.          -0.25881905 125.        ]
 [  0.           1.           0.           0.        ]
 [  0.25881905   0.           0.96592583  95.        ]
 [  0.           0.           0.           1.        ]]


In [56]:
theta = -1 * np.pi / 12

transformation_matrix3 = np.array([
    [ np.cos(theta), 0, np.sin(theta), -125],
    [            0, 1,            0, 0],
    [-np.sin(theta), 0, np.cos(theta), -95],
    [            0, 0,            0, 1]
])
print(transformation_matrix3)

[[   0.96592583    0.           -0.25881905 -125.        ]
 [   0.            1.            0.            0.        ]
 [   0.25881905    0.            0.96592583  -95.        ]
 [   0.            0.            0.            1.        ]]


In [87]:
x,y,z,rx,ry,rz = arm.get_position(is_radian=False)[1]
transform_base = np.eye(4)
transform_base[:3,:3] = R.from_euler('xyz', [rx,ry,rz], degrees=False).as_matrix()
transform_base[:3,3] = [x,y,z]

In [110]:
def create_transform(vec):
    x,y,z,rx,ry,rz = vec
    transform = np.eye(4)
    transform[:3,:3] = R.from_euler('xyz', [rx,ry,rz], degrees=True).as_matrix()
    transform[:3,3] = [x,y,z]
    return transform

In [119]:
def transform_to_vec(transform):
    x,y,z = transform[:3,3]
    rx,ry,rz = R.from_matrix(transform[:3,:3]).as_euler('xyz', degrees=True)
    return [x,y,z,rx,ry,rz]

In [123]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# iPhone is 125cm in front of the wrist, 95cm above the wrist, and tilted 15 degrees ccw about the y-axis
wrist_to_iphone = create_transform([125,0,95,0,-15,0])

for i in range(1):
    
    code, current_pos = arm.get_position(is_radian=False)
    base_to_end_effector = create_transform(current_pos)
    
    action = [0,0,0,0,-10,0]
    iphone_to_action = create_transform(action)
    
    full_transform = base_to_end_effector @ wrist_to_iphone @ iphone_to_action @ np.linalg.inv(wrist_to_iphone)
    
    new_pos = transform_to_vec(full_transform)
    
    arm.set_position(*new_pos, speed=100, mvacc=1000, wait=True)



In [102]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# prev_pos = np.zeros(6)

for i in range(1):
    # x,y,z,rx,ry,rz = prev_pos
    
    code, current_pos = arm.get_position(is_radian=False)
    x,y,z,rx,ry,rz = current_pos
    transform_base = np.eye(4)
    transform_base[:3,:3] = R.from_euler('xyz', [rx,ry,rz], degrees=True).as_matrix()
    transform_base[:3,3] = [x,y,z]

    # r = R.from_euler('xyz', [rx,ry,rz], degrees=True).as_matrix()
    # transformation0 = np.eye(4)
    # transformation0[:3,:3] = r
    # transformation0[:3,3] = [x,y,z]

    transformation_matrix2 = np.eye(4)
    transformation_matrix2[:3, :3] = R.from_euler('xyz', [0,10,0], degrees=True).as_matrix()
    # transformation_matrix2[2, 3] = 200
    
    full_transform = transform_base @ transformation_matrix1 @ transformation_matrix2 @ np.linalg.inv(transformation_matrix1)
    
    r = R.from_matrix(full_transform[:3,:3]).as_euler('xyz', degrees=True)
    x,y,z = full_transform[:3,3]
    
    new_pos = [x,y,z] + list(r)
    print("new absolute pos:", new_pos)
    # new_pos = list(np.array(new_pos) - np.array(prev_pos))
    print("prev pos:", prev_pos)
    print("new relative pos:", new_pos)

    prev_pos = new_pos
    
    arm.set_position(*new_pos, speed=100, mvacc=1000, wait=True)



Current position: [305.518585, 9.993493, 541.88092, -43.414158, -87.419379, -136.354903]
new absolute pos: [329.13095159543514, 10.52866022128222, 528.055309116347, -8.555834656801142, -77.99504020534845, -171.36864272544483]
prev pos: [329.13095159543514, 10.52866022128222, 528.055309116347, -8.555834656801142, -77.99504020534845, -171.36864272544483]
new relative pos: [329.13095159543514, 10.52866022128222, 528.055309116347, -8.555834656801142, -77.99504020534845, -171.36864272544483]
Current position: [329.130951, 10.52866, 528.055298, -8.555807, -77.995026, -171.36864]
new absolute pos: [354.7870883524995, 10.9397180785677, 518.5383959881126, -4.749927779000746, -68.05657167966598, -175.33278885000254]
prev pos: [329.13095159543514, 10.52866022128222, 528.055309116347, -8.555834656801142, -77.99504020534845, -171.36864272544483]
new relative pos: [354.7870883524995, 10.9397180785677, 518.5383959881126, -4.749927779000746, -68.05657167966598, -175.33278885000254]


In [93]:
code, current_pos = arm.get_position(is_radian=False)

In [95]:
arm.set_position(*current_pos, speed=100, mvacc=1000, wait=True)

0

In [106]:
new_pos = [27.202922734561554, 67.9903810567666, -7.289001178492001, -10.574001115377373, -4.199144429757126, -43.223900989167845]
arm.set_position(*new_pos, speed=100, mvacc=1000, wait=True, relative=True)

0

In [107]:
x,y,z,rx,ry,rz = new_pos
r = R.from_euler('xyz', [rx,ry,rz], degrees=True).as_matrix()
transformation0 = np.eye(4)
transformation0[:3,:3] = r
transformation0[:3,3] = [x,y,z]

transformation0 = np.linalg.inv(transformation0)
r = R.from_matrix(transformation0[:3,:3]).as_euler('xyz', degrees=True)
x,y,z = transformation0[:3,3]
new_pos = [x,y,z] + list(r)
# new_pos2 = list(-1*np.array(new_pos))
arm.set_position(*new_pos, speed=100, mvacc=1000, wait=True, relative=True)

0

In [49]:
code, current_pos = arm.get_position(is_radian=False)
print(f"Current position: {current_pos}")
x,y,z,rx,ry,rz = current_pos
transform_base = np.eye(4)
transform_base[:3,:3] = R.from_euler('xyz', [rx,ry,rz], degrees=True).as_matrix()
transform_base[:3,3] = [x,y,z]

transform_end_effector = np.eye(4)
transform_end_effector[:3,:3] = R.from_euler('xyz', [0,-20,0], degrees=True).as_matrix()

full_transform = transform_base @ transform_end_effector
r = R.from_matrix(full_transform[:3,:3]).as_euler('xyz', degrees=True)
x,y,z = full_transform[:3,3]

new_pos = [x,y,z] + list(r)

arm.set_position(*new_pos, speed=100, mvacc=1000, wait=True)
print("New Position:", new_pos)

Current position: [305.518585, 9.993493, 541.88092, -43.414158, -87.419379, -136.354903]
New Position: [305.518585, 9.993493, 541.88092, -174.31671173581006, -71.79153639334014, -5.140467676499437]


In [50]:
code, current_pos = arm.get_position(is_radian=False)
print(f"Current position: {current_pos}")
new_pos = np.zeros(6)
new_pos[4] = 20
arm.set_position(*list(new_pos), speed=100, mvacc=1000, wait=True, relative=True)
code, current_pos = arm.get_position(is_radian=False)
print(f"Current position: {current_pos}")

Current position: [305.518585, 9.993493, 541.88092, -43.414158, -87.419379, -136.354903]
Current position: [305.518585, 9.993493, 541.88092, -174.377069, -71.783991, -5.705227]


In [ ]:
arm.disconnect()